Bu tarz bir görev için çok popüler ve değerli olan gömme modelleri (embedding models) and vektör deposu (vector store) kullanılmaya başlanılacaktır.
Bir ürün kataloğunu ilgilenilen öğeler için sorgulamanıza izin veren bir araç buna örnek olarak verilebilir.

In [ ]:
#pip install --upgrade langchain
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # lokal .env dosyasını oku

from langchain.chains import RetrievalQA # soru cevap zinciri
from langchain.chat_models import ChatOpenAI # open ai chat modeli
from langchain.document_loaders import CSVLoader # csv dosyası yüklemek için
from langchain.vectorstores import DocArrayInMemorySearch # vektör deposu
from IPython.display import display, Markdown # bilgiyi göstermek için

In [ ]:
file = 'OutdoorClothingCatalog_1000.csv' # https://s172-31-4-92p30982.lab-aws-production.deeplearning.ai/edit/OutdoorClothingCatalog_1000.csv#
loader = CSVLoader(file_path=file)

In [ ]:
from langchain.indexes import VectorstoreIndexCreator # Bu indeks vektör deposu oluşturmayı kolaylaştırır

In [ ]:
#pip install docarray
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader]) # from_loaders doküman yükleyiciler listesi döner(bu örnekte 1 tane gönderiyoruz)

In [ ]:
query ="Lütfen güneş korumalı tüm gömleklerinizi indirimli bir tabloda \
listeleyin ve her birini özetleyin."

In [ ]:
response = index.query(query)
display(Markdown(response)) # Güneş korumalı gömlek isimleri ve açıklamaları ile birlikte en sonda bir özet tablo gösterir

In [ ]:
loader = CSVLoader(file_path=file)
docs = loader.load()
docs[0]

LLM'ler bir anda yalnızca birkaç bin kelimeyi inceleyebilir (parça(chunk)halinde), çok fazla dokümanımız varken soru cevap yapacak bir LLM'i nasıl sağlarız? İşte burada devreye gömme modelleri ve vektör depoları girer.
Gömme denilen işlemde kelimeler onları temsil edecek ve semantik (anlamsal) değerlerini tutacak vektörler (-0.003530 gibi) ile değiştirilirler. Doküman chunk denen parçalara bölünür ve o chunk'lardaki metinler vektöre embedding yöntemi ile çevrilir ve bunlar vektör veritabanına kaydedilir. Böylece bir prompt sorgusu geldiğinde o sorgu embed'e çevirlir ve tüm embedded chunk'lar içinde veritabanında aratılır ve n en benzer chunk içeriden alınıp dönülür. Bu dönülen değerler artık LLM bağlamına sığabilir ve böylece modelden çıktımızı alabiliriz.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

embed = embeddings.embed_query("Merhaba benim adım Harrison")

print(len(embed)) # 1536 embed vardır

print(embed[:5])  # Her bir embed vektör tutar

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, # doküman listesi
    embeddings # Embedding nesnesi
)

In [ ]:
query = "Lütfen güneşi önleyen bir gömlek önerin"

In [ ]:
docs = db.similarity_search(query) # Benzerliğe göre vektör deposunda arama yapar böylece dokümanları alırız

len(docs) # Doküman sayısı

docs[0]   # İlk doküman

Alıcı (retriever), belgeleri döndüren bir sorguyu alan herhangi bir yöntemle desteklenebilen genel bir arabirimdir. Vektör depoları ve gömme modellerinden başka daha karmaşık veya basit yöntemler de vardır.

In [ ]:
retriever = db.as_retriever()

llm = ChatOpenAI(temperature = 0.0)

# Dokümanları tek bir metin parçasına birleştiriyoruz
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

response = llm.call_as_llm(f"{qdocs} Soru: Lütfen güneş korumalı tüm \
gömleklerinizi bir tablo halinde listeleyin ve her birini özetleyin.")

display(Markdown(response))

# Stuffing (Doldurma)
Doldurma, dil modeline göndermek için tüm verileri bağlam biçiminde promptlara doldurduğumuz basit bir yöntemdir. Doldurma, LLM'e tek bir çağrı yapmayı mümkün kılar. LLM'in tüm verilere aynı anda erişimi vardır ancak LLM'lerin bir bağlam uzunluğu vardır. Büyük veya çok sayıda belge için bu, bağlamdan daha büyük bir promptla sonuçlanacağı için çalışmaz. Anlaması kolay, yeterince iyi çalışan, maaliyetsiz bir yöntemdir ancak her zaman iyi çalışmaz örneğin aynı tarz soru yanıtlama işlemini birçok chunk üzerinde yapmak istiyorsak ne yapacağız?
Doldurma yöntemi en popüleri olsa da kullanabileceğimiz 3 çeşit başka yöntem daha bulunmaktadır:


1.   **Map_reduce (harita indirgeme)**
Tüm chunk'ları alır ve soruyla birlikte LLM'e iletir. Ardından yanıtı alır ve tüm bireysel yanıtları nihai bir yanıtta özetlemek için başka bir dil model çağrısı kullanır. Bu gerçekten güçlü bir yöntemdir çünkü herhangi bir sayıda belge üzerinde kullanabilirsiniz. Chunkların her biri ayrı ayrı LLM'e soru yöneltir, bundan dolayı da çok daha fazla çağrı gerektirir ve her belgeyi ayrı ayrı ele almak çok arzu edilen bir yöntem değildir. Özetleme için tercih edilen bir yöntemdir.

2.   **Refine (arıtma)**
Birçok dokümana bakmak için kullanılan başka bir yöntemdir, bir önceki dokümandan alınan cevap üzerine kurularak çalışır. Zamanla cevapları farklı dokümanlardan toplayıp kombine etmek için uygun bir yöntemdir. Genellikle daha uzun cevapların daha yavaş sürede alınmasına neden olan bir yöntemdir. Yavaştır çünkü artık çağrılar bağımsızdır ve bir önceki çağrının sonucuna bağımlı oldukları için onları beklerler.

3. **Map_rerank (harita yeniden sıralaması)**
Tüm çağrıların bağımsız olduğu bu yöntem çok daha deneysel ve ilginçtr. Her doküman için LLM'e tek bir çağrı atılır ve skor dönmesi istenir. Ve en yüksek skorlu olan seçilir.


In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

query =  "Lütfen güneş korumalı tüm gömleklerinizi indirimli \
bir tabloda listeleyin ve her birini özetleyin."

response = qa_stuff.run(query)

display(Markdown(response))

response = index.query(query, llm=llm)

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])